Laboratorio 8


Redes

Sofi Lam 21548

In [ ]:
!pip install kafka-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 kB 4.4 MB/s eta 0:00:00


**3.1 Simulación de un Sensor**

In [ ]:
import random
import json

WIND_DIRECTIONS = ["N", "NO", "O", "SO", "S", "SE", "E", "NE"]

def generate_temperature(mean=25.0, std=5.0):

    t = random.gauss(mean, std)
    t = max(0.0, min(110.0, t))
    return round(t, 2)

def generate_humidity(mean=60.0, std=15.0):

    h = random.gauss(mean, std)
    h = int(round(h))
    h = max(0, min(100, h))
    return h

def generate_wind_direction():

    return random.choice(WIND_DIRECTIONS)

def generate_measurement():

    data = {
        "temperatura": generate_temperature(),
        "humedad": generate_humidity(),
        "direccion_viento": generate_wind_direction()
    }
    return data

def generate_measurement_json():

    meas = generate_measurement()
    return json.dumps(meas)


if __name__ == "__main__":
    for _ in range(10):
        m = generate_measurement()
        print("Diccionario:", m)
        print("JSON      :", json.dumps(m))
        print("-" * 40)


Diccionario: {'temperatura': 13.64, 'humedad': 23, 'direccion_viento': 'SE'}
JSON      : {"temperatura": 13.64, "humedad": 23, "direccion_viento": "SE"}
----------------------------------------
Diccionario: {'temperatura': 25.58, 'humedad': 67, 'direccion_viento': 'E'}
JSON      : {"temperatura": 25.58, "humedad": 67, "direccion_viento": "E"}
----------------------------------------
Diccionario: {'temperatura': 24.48, 'humedad': 89, 'direccion_viento': 'NE'}
JSON      : {"temperatura": 24.48, "humedad": 89, "direccion_viento": "NE"}
----------------------------------------
Diccionario: {'temperatura': 30.89, 'humedad': 42, 'direccion_viento': 'N'}
JSON      : {"temperatura": 30.89, "humedad": 42, "direccion_viento": "N"}
----------------------------------------
Diccionario: {'temperatura': 27.78, 'humedad': 77, 'direccion_viento': 'SE'}
JSON      : {"temperatura": 27.78, "humedad": 77, "direccion_viento": "SE"}
----------------------------------------



**3.2 Envío de Datos al Server Edge:**

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    bootstrap_servers="iot.redesuvg.cloud:9092",
)

print("Topics disponibles en el cluster:")
print(consumer.topics())


Topics disponibles en el cluster:
{'21333', '21403', '22305', '22266', 'estacion-meteorologica', '22243', '221038', '22787', '22272', '20201234', '2020XXXX', '22933', '22764-22249-compact', 'mnovellatest', '22075', '22087', '22376', '22513', '21548', '22386', '22944', '22801', '22404', '9092', '22982', 'jayass', '22716', '22500', '22102', '202012345', '22611', '22049', 'prueba-12345', '338800', '221038_bytes', '22295', '22386_1', '22153', 'TU_CARNET', '1234567', '21231', '22764-22249', '22873', '22398', '22296'}


In [ ]:
import json
import time
import random
from kafka import KafkaProducer

BOOTSTRAP_SERVERS = "iot.redesuvg.cloud:9092"
TOPIC = "21548"

producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

print(f"Productor iniciado. Enviando datos al topic {TOPIC}...")

try:
    while True:
        data = generate_measurement()

        print("Enviando:", data)
        producer.send(TOPIC, value=data)
        producer.flush()

        wait = random.randint(15, 30)
        print(f"Esperando {wait} segundos...\n")
        time.sleep(wait)

except KeyboardInterrupt:
    print("Productor detenido manualmente.")

finally:
    producer.close()


Productor iniciado. Enviando datos al topic 21548...
Enviando: {'temperatura': 25.38, 'humedad': 77, 'direccion_viento': 'NE'}
Esperando 26 segundos...

Enviando: {'temperatura': 21.63, 'humedad': 73, 'direccion_viento': 'E'}
Esperando 27 segundos...

Enviando: {'temperatura': 18.13, 'humedad': 40, 'direccion_viento': 'E'}
Esperando 18 segundos...

Enviando: {'temperatura': 22.42, 'humedad': 73, 'direccion_viento': 'NO'}
Esperando 30 segundos...

Enviando: {'temperatura': 27.06, 'humedad': 72, 'direccion_viento': 'N'}
Esperando 28 segundos...

Productor detenido manualmente.


**3.3 Consumir y Desplegar Datos Meteorológicos:**

In [ ]:
from kafka import KafkaConsumer
import json

BOOTSTRAP_SERVERS = "iot.redesuvg.cloud:9092"
TOPIC = "21548"

consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset="earliest",
    enable_auto_commit=False,
    group_id="grupo_21548_pruebas",
    value_deserializer=lambda x: json.loads(x.decode("utf-8")),
)

print(f"Consumidor escuchando en el topic {TOPIC}...")

try:
    for msg in consumer:
        print("Mensaje recibido:")
        print(msg.value)
        print("-" * 40)
except KeyboardInterrupt:
    print("Consumidor detenido.")


Consumidor escuchando en el topic 21548...
Mensaje recibido:
{'temperatura': 23.3, 'humedad': 73, 'direccion_viento': 'NO'}
----------------------------------------
Mensaje recibido:
{'temperatura': 29.13, 'humedad': 59, 'direccion_viento': 'N'}
----------------------------------------
Mensaje recibido:
{'temperatura': 25.48, 'humedad': 65, 'direccion_viento': 'SE'}
----------------------------------------
Mensaje recibido:
{'temperatura': 27.22, 'humedad': 87, 'direccion_viento': 'NO'}
----------------------------------------
Mensaje recibido:
{'temperatura': 27.21, 'humedad': 46, 'direccion_viento': 'SO'}
----------------------------------------
Mensaje recibido:
{'temperatura': 17.94, 'humedad': 48, 'direccion_viento': 'NE'}
----------------------------------------
Mensaje recibido:
{'temperatura': 13.84, 'humedad': 49, 'direccion_viento': 'N'}
----------------------------------------
Mensaje recibido:
{'temperatura': 24.49, 'humedad': 35, 'direccion_viento': 'E'}
------------------

**IoT en Entornos con Restricciones:**


In [23]:
from kafka import KafkaProducer
import json, random, time

BOOTSTRAP_SERVERS = "iot.redesuvg.cloud:9092"
TOPIC = "21548"

producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer = lambda v: json.dumps(v).encode("utf-8")
)

WIND_DIRECTIONS = ["N", "NO", "O", "SO", "S", "SE", "E", "NE"]

def generate_measurement():
    temp = max(0, min(110, round(random.gauss(25, 5), 2)))
    hum = max(0, min(100, int(round(random.gauss(60, 15)))))
    wind = random.choice(WIND_DIRECTIONS)
    return {"temperatura": temp, "humedad": hum, "direccion_viento": wind}

print("Enviando 5 mensajes de prueba a 21548...\n")

for i in range(5):
    data = generate_measurement()
    print(f"{i+1}.", data)
    producer.send(TOPIC, value=data)
    producer.flush()
    time.sleep(1)

producer.close()
print("\nListo, producer de prueba terminó.")


Enviando 5 mensajes de prueba a 21548...

1. {'temperatura': 20.53, 'humedad': 49, 'direccion_viento': 'O'}
2. {'temperatura': 26.88, 'humedad': 67, 'direccion_viento': 'S'}
3. {'temperatura': 20.84, 'humedad': 41, 'direccion_viento': 'N'}
4. {'temperatura': 25.67, 'humedad': 28, 'direccion_viento': 'SE'}
5. {'temperatura': 22.72, 'humedad': 69, 'direccion_viento': 'NO'}

Listo, producer de prueba terminó.


In [24]:
from kafka import KafkaConsumer, KafkaProducer
import json
from datetime import datetime

BOOTSTRAP_SERVERS = "iot.redesuvg.cloud:9092"

TOPIC_ORIGEN = "21548"
TOPIC_DESTINO = "processed-21548"

producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

consumer = KafkaConsumer(
    TOPIC_ORIGEN,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    group_id="procesador-21548",
    value_deserializer=lambda x: json.loads(x.decode("utf-8")),
)

print(f"Procesador escuchando en {TOPIC_ORIGEN} y enviando a {TOPIC_DESTINO}...\n")

try:
    for msg in consumer:
        data = msg.value

        procesado = {
            "temperatura": data["temperatura"],
            "humedad": data["humedad"],
            "direccion_viento": data["direccion_viento"],
            "timestamp_procesado": datetime.now().isoformat(),
            "clima": "caluroso" if data["temperatura"] >= 28 else "templado"
        }

        print("Mensaje recibido:")
        print(data)
        print(" Publicando procesado:")
        print(procesado)
        print("-" * 40)

        producer.send(TOPIC_DESTINO, value=procesado)
        producer.flush()

except KeyboardInterrupt:
    print("Procesador detenido.")
finally:
    producer.close()


Procesador escuchando en 21548 y enviando a processed-21548...

Mensaje recibido:
{'temperatura': 20.84, 'humedad': 41, 'direccion_viento': 'N'}
 Publicando procesado:
{'temperatura': 20.84, 'humedad': 41, 'direccion_viento': 'N', 'timestamp_procesado': '2025-11-17T01:45:47.522024', 'clima': 'templado'}
----------------------------------------
Mensaje recibido:
{'temperatura': 25.67, 'humedad': 28, 'direccion_viento': 'SE'}
 Publicando procesado:
{'temperatura': 25.67, 'humedad': 28, 'direccion_viento': 'SE', 'timestamp_procesado': '2025-11-17T01:45:47.646933', 'clima': 'templado'}
----------------------------------------
Mensaje recibido:
{'temperatura': 20.53, 'humedad': 49, 'direccion_viento': 'O'}
 Publicando procesado:
{'temperatura': 20.53, 'humedad': 49, 'direccion_viento': 'O', 'timestamp_procesado': '2025-11-17T01:45:47.682335', 'clima': 'templado'}
----------------------------------------
Mensaje recibido:
{'temperatura': 22.72, 'humedad': 69, 'direccion_viento': 'NO'}
 Publi